In [1]:
import sys
import os
sys.path.append(os.getcwd())
import init


STBX not loaded: No module named 'snappy'
init loaded


In [2]:
if init.ENVIRONMENT == 'blaze':
    import subprocess

    command = 'source /usr/local/cuda/CUDA_VISIBILITY.csh'
    process = subprocess.Popen(command, shell=True, executable="/bin/csh", stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    stdout, stderr = process.communicate()

    os.environ['CUDA_VISIBLE_DEVICES'] = stdout.decode()[-2]
    # os.environ['CUDA_HOME'] = '/opt/cuda/cuda-10.0'
    print(stdout.decode())

    command = 'source /server/opt/cuda/enable_cuda_11.0'
    process = subprocess.Popen(command, shell=True, executable="/bin/csh", stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    stdout, stderr = process.communicate()

    command = 'echo $CUDA_VISIBLE_DEVICES'
    process = subprocess.Popen(command, shell=True, executable="/bin/csh", stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    stdout, stderr = process.communicate()
    print(command)
    print(stdout.decode())


In [3]:
import tensorflow as tf
import time
import importlib

from dataset.reader import Reader
import sis_toolbox as tbx


2023-07-13 12:11:20.145988: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-13 12:11:20.220753: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [4]:
init.setup_output()


In [5]:
# Dynamically import all classes in the directory
directory = 'models'
modules = []
for filename in os.listdir(directory):
    if filename.endswith('.py'):
        module_name = filename[:-3]
        modules.append(importlib.import_module("." + module_name, package=directory))

model = None
for module in modules:
    if module.__name__[-len(init.MODEL_NAME):] == init.MODEL_NAME:
        model = module.GAN()


2023-07-13 12:11:33.132697: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [6]:
generator = model.generator
discriminator = model.discriminator


In [8]:
req_tf_version = '2.8.0'
tf_gtet_280 = True
for reqPart, part in zip(map(int, req_tf_version.split(".")), map(int, tf.__version__.split("."))):
    if reqPart > part:
        tf_gtet_280 = False
        break
    if reqPart < part:
        break

if tf_gtet_280:
    tf.keras.utils.plot_model(generator, show_shapes=True, expand_nested=True, show_layer_activations=True, to_file=os.path.join(init.OUTPUT_MODEL, 'generator.png'))
    tf.keras.utils.plot_model(discriminator, show_shapes=True, expand_nested=True, show_layer_activations=True, to_file=os.path.join(init.OUTPUT_MODEL, 'discriminator.png'))
else:
    tf.keras.utils.plot_model(generator, show_shapes=True, expand_nested=True, to_file=os.path.join(init.OUTPUT_MODEL, 'generator.png'))
    tf.keras.utils.plot_model(discriminator, show_shapes=True, expand_nested=True, to_file=os.path.join(init.OUTPUT_MODEL, 'discriminator.png'))
None


In [9]:
dataset_reader = Reader(init.BATCH_SIZE, init.SHUFFLE, 'train.ipynb', init.DATA_SAMPLE)
train_dataset = dataset_reader.train_dataset
test_dataset = dataset_reader.test_dataset


datamodel.Reader called by train.py
selected random sample train: 25000
selected random sample val: 5000


In [10]:
def fit(train_ds, test_ds, steps):
    # example_target, example_input = next(iter(test_ds.take(1)))
    start = time.time()
    example_targets = []
    example_inputs = []
    for example_target, example_input in test_dataset.take(5):
        example_targets.append(example_target[0])
        example_inputs.append(example_input[0])
    example_inputs = tf.stack(example_inputs, axis=0)
    example_targets = tf.stack(example_targets, axis=0)

    for step, (target, input_image) in train_ds.repeat().take(steps).enumerate():
        if step % init.SAMPLE_FREQ == 0:
            # display.clear_output(wait=True)
            
            if step != 0:
                print(f'Time taken for {init.SAMPLE_FREQ} steps: {time.time()-start:.2f} sec\n')
                start = time.time()
            
            tbx.generate_images(generator, example_inputs, example_targets, showimg=False, PATH_IMGS=init.OUTPUT_SAMPLES, savemodel=init.MODEL_NAME, iteration=step)
            # for example_target, example_input in test_dataset.take(1):
            #     helper.generate_images(generator, example_input, example_target, showimg=False, PATH_IMGS=path_imgs, savemodel=model.name, starttimestamp=STARTTIME, iteration=step)

            print(f"Step: {step}")

        model.train_step(input_image, target, step)

        # Training step
        if (step + 1) % 10 == 0:
            print('.', end='', flush=True)

        # Save (checkpoint) the model every 5k steps
        if (step + 1) % init.CKPT_FREQ == 0:
            print(f'Step + 1 = {step + 1} - saving checkpoint')
            model.save()


In [11]:
fit(train_dataset, test_dataset, steps=init.STEPS)


2023-07-13 12:12:41.947690: I tensorflow/core/grappler/optimizers/data/replicate_on_split.cc:32] Running replicate on split optimization
2023-07-13 12:12:52.030028: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:392] Filling up shuffle buffer (this may take a while): 81 of 2500
2023-07-13 12:13:02.038507: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:392] Filling up shuffle buffer (this may take a while): 166 of 2500


2023-07-13 12:13:12.020258: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:392] Filling up shuffle buffer (this may take a while): 258 of 2500


#### Visualizations / not for train.py

In [ ]:
import random

sample_dataset = tf.data.TFRecordDataset(os.path.join(init.VAL_DIR, random.choice(os.listdir(init.VAL_DIR))))
for element in sample_dataset:
    tbx.plot_tensor_sbs(element, init.TILESIZE)
    s2_tensor, s3_tensor = tbx.parse_tfrecord(element, init.TILESIZE)
    s2_tensor, s3_tensor = dataset_reader.resize(s2_tensor, s3_tensor, init.IMG_HEIGHT, init.IMG_WIDTH)



In [ ]:
from sis_toolbox import RGBProfile as rgb

gen_output = generator(s3_tensor[tf.newaxis, ...], training=False)
tbx.plot_tensor(gen_output[0], rgb.S2)


In [ ]:
import matplotlib.pyplot as plt

disc_out = discriminator([s3_tensor[tf.newaxis, ...], gen_output], training=False)
plt.imshow(disc_out[0, ..., -1], vmin=-20, vmax=20, cmap='RdBu_r')
plt.colorbar()


In [ ]:
example_targets = []
example_inputs = []
for example_target, example_input in test_dataset.take(5):
    example_targets.append(example_target[0])
    example_inputs.append(example_input[0])

tbx.generate_images(generator, tf.stack(example_inputs, axis=0), tf.stack(example_targets, axis=0))


In [ ]:
# !tensorboard dev upload --logdir {PATH_LOGS}